In [1]:
import requests
import pprint

In [2]:
january2021 = requests.get("http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=4597174352ee463084a193409241305&q=London&format=json&date=2021-01-01&enddate=2021-01-31") 
print(january2021.status_code)


200


In [3]:
february2021 = requests.get("http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=4597174352ee463084a193409241305&q=London&format=json&date=2021-02-01&enddate=2021-02-28") 
print(february2021.status_code)

200


In [4]:
march2021 = requests.get("http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=4597174352ee463084a193409241305&q=London&format=json&date=2021-03-01&enddate=2021-03-31") 
print(march2021.status_code)

200


In [ ]:
march2022 = requests.get("http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=4597174352ee463084a193409241305&q=London&format=json&date=2022-03-01&enddate=2022-03-31") 
print(march2022.status_code)

In [5]:
stuff = [january2021.json()]

stuff.insert(0, stuff)
pprint.pp(stuff)

[<Recursion on list with id=4417606848>,
 {'data': {'request': [{'type': 'City', 'query': 'London, United Kingdom'}],
           'weather': [{'date': '2021-01-01',
                        'astronomy': [{'sunrise': '08:06 AM',
                                       'sunset': '04:02 PM',
                                       'moonrise': '06:26 PM',
                                       'moonset': '10:03 AM',
                                       'moon_phase': 'Waning Gibbous',
                                       'moon_illumination': '79'}],
                        'maxtempC': '3',
                        'maxtempF': '38',
                        'mintempC': '-1',
                        'mintempF': '31',
                        'avgtempC': '1',
                        'avgtempF': '34',
                        'totalSnow_cm': '0.0',
                        'sunHour': '4.0',
                        'uvIndex': '2',
                        'hourly': [{'time': '0',
                     

In [6]:

stuff2 = [february2021.json()]

stuff2.insert(0, stuff2)
pprint.pp(stuff2)

[<Recursion on list with id=4419207488>,
 {'data': {'request': [{'type': 'City', 'query': 'London, United Kingdom'}],
           'weather': [{'date': '2021-02-01',
                        'astronomy': [{'sunrise': '07:39 AM',
                                       'sunset': '04:50 PM',
                                       'moonrise': '09:31 PM',
                                       'moonset': '09:42 AM',
                                       'moon_phase': 'Waning Gibbous',
                                       'moon_illumination': '63'}],
                        'maxtempC': '6',
                        'maxtempF': '42',
                        'mintempC': '4',
                        'mintempF': '39',
                        'avgtempC': '5',
                        'avgtempF': '40',
                        'totalSnow_cm': '0.0',
                        'sunHour': '1.0',
                        'uvIndex': '1',
                        'hourly': [{'time': '0',
                      

In [7]:
stuff3 = [march2021.json()]

stuff3.insert(0, stuff3)
pprint.pp(stuff3)

[<Recursion on list with id=4419809344>,
 {'data': {'request': [{'type': 'City', 'query': 'London, United Kingdom'}],
           'weather': [{'date': '2021-03-01',
                        'astronomy': [{'sunrise': '06:45 AM',
                                       'sunset': '05:41 PM',
                                       'moonrise': '08:35 PM',
                                       'moonset': '08:04 AM',
                                       'moon_phase': 'Waning Gibbous',
                                       'moon_illumination': '79'}],
                        'maxtempC': '10',
                        'maxtempF': '50',
                        'mintempC': '5',
                        'mintempF': '40',
                        'avgtempC': '7',
                        'avgtempF': '44',
                        'totalSnow_cm': '0.0',
                        'sunHour': '11.0',
                        'uvIndex': '3',
                        'hourly': [{'time': '0',
                    